# Disparity estimation in OpenCV

OpenCV already includes implementations of a few disparity estimation algorithms, which are shown below.

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
PLY_HEADER = '''ply
format ascii 1.0
element vertex %(vert_num)d
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
end_header
'''

def write_ply(fn, verts, colors):
    verts = verts.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    verts = np.hstack([verts, colors])
    with open(fn, 'wb') as f:
        f.write((PLY_HEADER % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts, fmt='%f %f %f %d %d %d ')

In [3]:
imgL = cv2.pyrDown( cv2.imread('./data/view_left.png') )  # downscale images for faster processing
imgR = cv2.pyrDown( cv2.imread('./data/view_right.png') )

# disparity range is tuned for 'aloe' image pair
window_size = 3
min_disp = 16
num_disp = 112 - min_disp

stereo = cv2.StereoSGBM_create(minDisparity = min_disp,
    numDisparities = num_disp,
    blockSize = 16,
    P1 = 8*3*window_size**2,
    P2 = 32*3*window_size**2,
    disp12MaxDiff = 1,
    uniquenessRatio = 10,
    speckleWindowSize = 100,
    speckleRange = 32
)

disparity = stereo.compute(imgL, imgR).astype(np.float32) / 16.0

h, w = imgL.shape[:2]
f = 0.8*w                          # guess for focal length
Q = np.float32([[1, 0, 0, -0.5*w],
                [0,-1, 0,  0.5*h], # turn points 180 deg around x-axis,
                [0, 0, 0,     -f], # so that y-axis looks up
                [0, 0, 1,      0]])
points = cv2.reprojectImageTo3D(disparity, Q)
colors = cv2.cvtColor(imgL, cv2.COLOR_BGR2RGB)

mask = disparity > disparity.min()
out_points = points[mask]
out_colors = colors[mask]

out_fn = 'dip_est_cv2_out.ply'
write_ply(out_fn, out_points, out_colors)

In [4]:
import open3d as o3d

pcd = o3d.io.read_point_cloud(out_fn)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
from lib.plots import plot_point_cloud

plot = plot_point_cloud(pcd, point_size=0.1)

plot.camera = [
    5.4, 2.0, 4.0,
    0, 0, -18,
    0, 1, 0
]

plot.display()

Output()